In [1]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torchvision import datasets
from torchvision import transforms

import numpy as np

import optuna
from optuna.trial import TrialState

/home/jfialho/.conda/envs/ml_default/lib/python3.7/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [12]:
DEVICE = torch.device("cuda:1")
BATCHSIZE = 128
CLASSES = 10
DIR = os.getcwd()
EPOCHS = 10
LOG_INTERVAL = 10
N_TRAIN_EXAMPLES = BATCHSIZE * 30
N_VALID_EXAMPLES = BATCHSIZE * 10

In [13]:
# hyperparameters set
range_opt_name = ["Adam", "RMSprop", "SGD"]
range_lr = [1e-5, 1e-1]
range_n_layers = [1, 4]
range_p = [0.15, 0.5]
range_out_features = [2**2, 2**8]

In [14]:
def objective(trial):    
    layers = []
    acc = []

    # Generate the model.
    in_features = 28 * 28
    n_layers = trial.suggest_int("n_layers", range_n_layers[0], range_n_layers[1])
    
    for i in range(n_layers): 
        out_features = trial.suggest_int("n_units_l{}".format(i),range_out_features[0], range_out_features[1])
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        p = trial.suggest_float("dropout_l{}".format(i), range_p[0], range_p[1])
        layers.append(nn.Dropout(p))
        
        in_features = out_features
    layers.append(nn.Linear(in_features, CLASSES))
    layers.append(nn.LogSoftmax(dim=1))

    model =  nn.Sequential(*layers).to(DEVICE)


    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer",range_opt_name)
    lr = trial.suggest_float("lr",range_lr[0],range_lr[1],log=True)
    
    optimizer = getattr(optim, optimizer_name)(model.parameters(),lr=lr)

    # Load FashionMNIST dataset.
    train_loader = torch.utils.data.DataLoader(
        datasets.FashionMNIST(DIR, train=True, download=True, transform=transforms.ToTensor()),
        batch_size=BATCHSIZE,
        shuffle=True,
    )
    valid_loader = torch.utils.data.DataLoader(
        datasets.FashionMNIST(DIR, train=False, transform=transforms.ToTensor()),
        batch_size=BATCHSIZE,
        shuffle=True,
    )


    # Training of the model.
    for epoch in range(EPOCHS):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            # Limiting training data for faster epochs.
            if batch_idx * BATCHSIZE >= N_TRAIN_EXAMPLES:
                break

            data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)

            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()

        # Validation of the model.
        model.eval()
        correct = 0
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(valid_loader):
                # Limiting validation data.
                if batch_idx * BATCHSIZE >= N_VALID_EXAMPLES:
                    break
                data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)
                output = model(data)
                # Get the index of the max log-probability.
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(target.view_as(pred)).sum().item()

        accuracy = correct / min(len(valid_loader.dataset), N_VALID_EXAMPLES)
        acc.append(accuracy)
        
        trial.report(accuracy, epoch)
        
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        #print("epoch:",epoch,"| acc:",round(accuracy*100,2),"%")

    #print best value of acc    
    #print("--------------------------")
    #print("best acc:", np.max(acc))
    
    return np.max(acc)
    

In [15]:
study = optuna.create_study(study_name='opt_tutorial',
                           storage='sqlite:///tutorial.db',
                           load_if_exists=True,
                           direction="maximize")

[I 2022-04-06 13:42:21,225] Using an existing study with name 'opt_tutorial' instead of creating a new one.


In [16]:
study.optimize(objective, n_trials=20,timeout=None)

[I 2022-04-06 13:42:28,029] Trial 5 pruned. 
[I 2022-04-06 13:42:38,561] Trial 6 finished with value: 0.1234375 and parameters: {'n_layers': 2, 'n_units_l0': 243, 'dropout_l0': 0.1861095884541992, 'n_units_l1': 217, 'dropout_l1': 0.36656273701135195, 'optimizer': 'SGD', 'lr': 0.0001369588221215791}. Best is trial 2 with value: 0.6078125.
[I 2022-04-06 13:42:49,263] Trial 7 finished with value: 0.67109375 and parameters: {'n_layers': 2, 'n_units_l0': 213, 'dropout_l0': 0.3245114110377013, 'n_units_l1': 14, 'dropout_l1': 0.25557082315982604, 'optimizer': 'RMSprop', 'lr': 0.009179785438385876}. Best is trial 7 with value: 0.67109375.
[I 2022-04-06 13:42:59,584] Trial 8 finished with value: 0.15078125 and parameters: {'n_layers': 1, 'n_units_l0': 8, 'dropout_l0': 0.41991061445385436, 'optimizer': 'SGD', 'lr': 4.048798071270917e-05}. Best is trial 7 with value: 0.67109375.
[I 2022-04-06 13:43:10,813] Trial 9 finished with value: 0.75703125 and parameters: {'n_layers': 3, 'n_units_l0': 225, 

In [17]:
trial = study.best_trial

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Best trial:")
print(" Value: ", trial.value)

print("\nStudy statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("\nBest params: ")
for key, value in trial.params.items():    
    print(" ",key,"=", value)

Best trial:
 Value:  0.828125

Study statistics: 
  Number of finished trials:  25
  Number of pruned trials:  4
  Number of complete trials:  21

Best params: 
  dropout_l0 = 0.29733833743419186
  dropout_l1 = 0.18955128033949867
  lr = 0.0015365378703487743
  n_layers = 2
  n_units_l0 = 123
  n_units_l1 = 114
  optimizer = RMSprop
